<a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![MAIA banner](https://raw.githubusercontent.com/MAIA4361-Aprendizaje-refuerzo-profundo/Notebooks_Tareas/main/Images/Aprendizaje_refuerzo_profundo_Banner_V1.png)

# <h1><center>Tarea Tutorial - Semana 6 <a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana6.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

<center><h1>PPO y TRPO</h1></center>

Este tutorial sigue con el estudio de algoritmos de gradiente de política. Concretamente, se revisarán 2 algoritmos: Trust Region Policy Optimization (TRPO) y Proximal Policy Optimization (PPO). Se utilizará la implementación de ambos algoritmos encontrada dentro de <a href="https://stable-baselines3.readthedocs.io/en/master/index.html">Stable Baselines3</a>, y los usaremos para resolver dos problemas de robótica basados en el entorno de simulación MuJoCo: <a href="https://gymnasium.farama.org/environments/mujoco/hopper/">Hopper</a> y <a href="https://gymnasium.farama.org/environments/mujoco/ant/">Ant</a>. Este notebook tutorial se divide en las siguientes secciones:


# Tabla de Contenidos
1. [Objetivos de Aprendizaje](#scrollTo=Objetivos_de_Aprendizaje)  
2. [Marco Teórico](#scrollTo=Marco_Te_rico)  
3. [Instalación de Librerías](#scrollTo=Instalaci_n_de_Librer_as)  
4. [Familiarización con los Entornos de Gym](#scrollTo=Familiarizaci_n_con_los_Entornos_de_Gym)  
5. [Hopper](#scrollTo=Hopper)  
6. [Ant](#scrollTo=Ant)  
7. [Reflexiones Finales](#scrollTo=Reflexiones_Finales)  
8. [Referencias](#scrollTo=Referencias)

# Objetivos de Aprendizaje  
  
* Comprender las bases teóricas detrás de algoritmos de gradiente de política basados en optimización restringida.
* Solucionar problemas de robótica basados en simulaciones físicas usando los algoritmos de TRPO y PPO.
* Comparar las ventajas y desventajas que existen entre TRPO y PPO.


# Marco Teórico  

Antes de explicar el funcionamiento concreto de los algoritmos de Trust Region Policy Optimization (TRPO) y Proximal Policy Optimization (PPO) es importante comprender el concepto de Divergencia de Kullback-Leibler, llamada abreviadamente como Divergencia KL. La Divergencia KL es una métrica que mide cuán diferente es una distribución de probabilidad de otra. Por ejemplo, si consideramos dos distribuciones de probabilidad genéricas $P_1(x)$ y $P_2(x)$, se define la Divergencia KL de la siguiente manera:

<center> $ D_{KL} = \sum_x P_1(x) \text{log}(\frac{P_1(x)}{P_2(x)}) $ &emsp;&emsp;&emsp;$(1)$ </center>

En la teoría del Aprendizaje por Refuerzo, la Divergencia KL se utiliza para comparar y medir la diferencia que existe entre una política antigua ($\pi_{old}$) y una política nueva ($\pi_{new}$). Una Divergencia KL pequeña indica que ambas políticas son parecidas, que no hubo grandes cambios en la actualización, mientras que una Divergencia KL grande indica que ambas políticas son muy diferentes entre sí.

En TRPO, si se denota al retorno descontado esperado como $\eta(\pi)$, y $L_{\pi_{old}}(\pi_{new})$ una aproximación local de $\eta$, se puede demostrar la siguiente cota:

<center> $ \eta(\pi_{new}) \geq L_{\pi_{old}}(\pi_{new}) - C \cdot D_{KL}^{max} (\pi_{old}, \pi_{new})$ &emsp;$ \text{donde} $ &emsp; $C = \frac{4 \epsilon \gamma}{(1-\gamma)^2} $&emsp;&emsp;&emsp;$(2)$ </center>

La Ecuación (2) muestra que mientras que la Divergencia KL se mantenga dentro de un límite, por debajo de un umbral pequeño $\delta$, se puede asegurar que el rendimiento no decrecerá, garantizando una mejora o por lo menos estabilidad. En pocas palabras, el algoritmo de TRPO busca solucionar el problema de optimización restringida dado por la Ecuación (3):

$$
\begin{aligned}
\text{maximize}_\theta  & \ \ L_{\theta_{old}}(\theta) \\
\text{subject to}  & \ \ D_{KL}^{max} (\theta_{old}, \theta) \leq \delta \qquad \text{(3)}
\end{aligned}
$$

Dado que también se puede demostrar la equivalencia $D_{KL} (\pi_{\theta_{old}}, \pi_{\theta}) = D_{KL} (\theta_{old}, \theta)$. Expandiendo el término $L_{\theta_{old}}$, se puede reescribir la Ecuación (4). En esta expresión, se utiliza la notación $\rho$ para definir la frecuencia de visitas y $\hat{A}$ la ventaja.

$$
\begin{aligned}
\text{maximize}_{\theta} \quad & \sum_s \rho_{\theta_{\text{old}}}(s) \sum_a \pi_{\theta}(a|s) \hat{A}_{\theta_{\text{old}}}(s, a) \\
\text{subject to} \quad & \bar{D}^{\rho^{\theta_{\text{old}}}}_{\mathrm{KL}}(\theta_{\text{old}}, \theta) \leq \delta \qquad \text{(4)}
\end{aligned}
$$

TRPO es en teoría fiable y robusto, pero en la práctica resulta difícil de implementar debido a este problema de optimización restringida, especialmente cuando la arquitectura de las redes nueronales se hace compleja. PPO ofrece una implementación más sencilla e igual o más estable que TRPO.

Primero, se define $r_t(\theta)$ como una razón de probabilidad:

<center> $ r_t(\theta) = \frac{\pi_{\theta}(a_t, s_t)}{\pi_{\theta_{old}}(a_t, s_t)} $ &emsp;&emsp;&emsp;$(5) $ </center>

Con esta definición, capaz de medir el cambio de probabilidad de una política respecto a la anterior, se puede también definir un objetivo "sustituto" de TRPO:

<center> $ L(\theta) = \hat{\mathbb{E}}_t \left[ \frac{\pi_{\theta}(a_t \mid s_t)}{\pi_{\theta_{\text{old}}}(a_t \mid s_t)} \hat{A}_t \right] = \hat{\mathbb{E}}_t \left[ r_t(\theta) \hat{A}_t \right] $ &emsp;&emsp;&emsp;$(6) $ </center>

La maximización del término $L(\theta)$ en la Ecuación (6) llevaría a una actualización demasiado grande de la política. Por ello, en PPO se considera cómo modificar este objetivo para penalizar cambios que muevan a $r_t(\theta)$ lejos de 1. La solución es aplicar un <i>clipping</i>, como se muestra en la Ecuación (7).

<center> $ L^{\text{CLIP}}(\theta) = \hat{\mathbb{E}}_t \left[ \min \left( r_t(\theta) \hat{A}_t,\ \text{clip}(r_t(\theta), 1 - \epsilon, 1 + \epsilon) \hat{A}_t \right) \right] $ &emsp;&emsp;&emsp;$(7) $ </center>

Donde $\epsilon$ es un hiperparámetro utilizado para modificar el rango de <i>clipping</i>. El primer término dentro de la función de minimización es la función $L(\theta)$ original de la Ecuación (6), y el segundo término modifica este valor recortando el rango de probabilidad, removiendo el incentivo de mover la razón $r_t(\theta)$ fuera del rango $[1-\epsilon,1+\epsilon]$. Al tomar el mínimo entre el objetivo original y recortado se obtiene un límite inferior más pesimista y estable. De esta manera, los cambios en la razón de probabilidades se ignora cuando el objetivo realmente mejora, y se tiene en cuenta cuando empeora. En resumen, el cambio es penalizado, no se sobreestima el beneficio y se tiene una aproximación pesimista más estable gracias al <i>clipping</i>.

# Instalación de Librerías  

Corra el siguiente bloque de código para instalar las librerías requeridas en el tutorial. Se encuentra incluído el módulo oficial de Stable Baselines3 que ejecuta A2C y SAC, y los ambientes de Gymnasium que incluyen Hopper y Ant. Esta celda puede tardar un par de minutos en ejecutarse completamente la primera vez.


In [ ]:

#Limpia los registros generados
from IPython.display import clear_output
clear_output()
print("Todas las librerías han sido instaladas correctamente.")

Todas las librerías han sido instaladas correctamente.


# Familiarización con los Entornos de Gym

Los siguientes ambientes de Gymnasium son entornos basados en MuJoCo de Google Deep Mind. MuJoCo (Multi-Joint dynamics with Contact) es un simulador físico utilizado para modelar y estudiar el comportamiento de sistemas dinámicos complejos, muy utilizado en robótica.

## Hopper

El ambiente de <a href="https://gymnasium.farama.org/environments/mujoco/hopper/">Hopper</a> consiste en una pierna en dos dimensiones que tiene como objetivo moverse hacia adelante modificando el torque de sus motores. Concretamente, la pierna se compone de cuatro partes: un torso en la parte superior, un muslo, una pantorrilla y un pie. Y cuenta con tres motores, uno para el muslo ubicado en la cadera, uno en la rodilla, y uno en el tobillo. Un diagrama esquemático del cuerpo y una imágen del entorno de simulación se muestran en la Figura 1.

![Action_space_hopper](https://raw.githubusercontent.com/MAIA4361-Aprendizaje-refuerzo-profundo/Notebooks_Tareas/main/Images/Action_space_hopper.png)

<center>Figura 1. Descripción del ambiente de Hopper. [4]</center>

Como se ve en la Figura 1, el espacio de acciones está descrito por el control de torque en los tres motores ubicados en las bisagras de la pierna. Los índices de los motores son respectivamente 0, 1 y 2, y el rango de control de cada motor se normaliza entre -1 y 1.

Por otro lado, el espacio de observación está descrito por 5 posiciones (<i>qpos</i>) y 6 velocidades (<i>qvel</i>) de las partes individuales del robot.

Finalmente, la recompensa es calculada ponderando 3 aspectos clave: la estabilidad del mecanismo por la cual recibe una recompensa de +1 por cada timestep en el cual la pierna no cae de cierta altura, la posición del torso que se usa para evaluar que la pierna avanza, y una penalización de control que castiga cuando se toman acciones muy grandes en los motores.  

## Ant

El ambiente de <a href="https://gymnasium.farama.org/environments/mujoco/ant/">Ant</a> consiste en un robot cuadrúpedo formado por un torso con forma esférica y cuatro piernas con 2 partes cada una. En total el robot cuenta con 8 motores, y el objetivo de esta "hormiga" es moverse hacia adelante (dirección derecha de la simulación).

![Action_space_hopper](https://raw.githubusercontent.com/MAIA4361-Aprendizaje-refuerzo-profundo/Notebooks_Tareas/main/Images/Action_space_ant.png)

<center>Figura 2. Descripción del ambiente de Ant. [5]</center>

De forma similar a Hopper, en Ant el espacio de acciones está dado por el control de torque entre -1 y 1 de los 8 motores del sistema dinámico de acuerdo con los índices de la Figura 2.

En este caso, el espacio de observación está descrito por 13 posiciones (<i>qpos</i>) y 14 velocidades (<i>qvel</i>) de las partes del robot, y 78 elementos adicionales relacionados con el centro de masa y fuerzas externas al sistema.

La recompensa es calculada ponderando 4 aspectos. Al igual que en Hopper, en Ant se le da una recompensa en cada timestep por conservar la estabilidad, también se recompensa que el robot avance en la dirección deseada, y se penalizan acciones muy grandes. El cuarto elemento involucrado en este ambiente corresponde a un castigo si las fuerzas de contacto externas son demasiado grandes.

In [ ]:
#

# =====================================================
# COMPLETAR ===========================================
#

# =====================================================

# Reflexiones Finales

Teniendo en cuenta los resultados observado en ambos ambientes con PPO y TRPO, reflexione sobre las siguientes preguntas:


*   ¿?

*   ¿?

*   ¿?

*   ¿?

# Referencias

[1] Sutton, R. S. and Barto, A. G. (2018). Reinforcement Learning: An Introduction. The MIT Press, second edition.

[2] Schulman, J., Levine S., Moritz P., Jordan M. and Abbeel P. (2015). Trust Region Policy Optimization. cite: arXiv:1502.05477

[3] Schulman, J., Wolski, F., Dhariwal, P., Radford, A. and O. Klimov. (2017). Proximal Policy Optimization Algorithms. cite: arXiv:1707.06347

[4] Gym Documentation, Hopper. `https://gymnasium.farama.org/environments/mujoco/hopper/`

[5] Gym Documentation, Ant. `https://gymnasium.farama.org/environments/mujoco/ant/`

[6] Stable Baselines3 Contrib Documentation, TRPO. `https://sb3-contrib.readthedocs.io/en/master/modules/trpo.html`

[7] Stable Baselines3 Documentation, PPO. `https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html`